# Complete Fitting Pipeline

This notebook demonstrates the full fitting pipeline:

1. **NLSQ** -- Fast point estimates with confidence intervals
2. **Bayesian NUTS** -- Full posterior via NumPyro with NLSQ warm-start
3. **Model comparison** -- Selecting between single and double exponential

## Prerequisites

- xpcsviewer installed
- For Bayesian fitting: `pip install numpyro jax arviz`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Generate Synthetic G2 Data

We create realistic synthetic data so this notebook is self-contained.

In [ ]:
np.random.seed(42)

# True parameters
tau_true = 5.0
baseline_true = 1.0
contrast_true = 0.3

# Generate delay times (log-spaced, typical for XPCS)
t = np.logspace(-2, 3, 80)

# Generate G2 data with noise
g2_clean = baseline_true + contrast_true * np.exp(-2 * t / tau_true)
noise_level = 0.005
g2_noise = g2_clean + np.random.normal(0, noise_level, size=t.shape)
g2_err = np.full_like(t, noise_level)

# Plot
fig, ax = plt.subplots()
ax.errorbar(t, g2_noise, yerr=g2_err, fmt='o', ms=4, capsize=2, label='Synthetic data')
ax.plot(t, g2_clean, 'r-', linewidth=2, label='True model', alpha=0.7)
ax.set_xscale('log')
ax.set_xlabel('Delay time (s)')
ax.set_ylabel(r'$g_2(\tau)$')
ax.set_title('Synthetic G2 Data')
ax.legend()
plt.tight_layout()
plt.show()

print(f"True parameters: tau={tau_true}, baseline={baseline_true}, contrast={contrast_true}")

## 2. NLSQ Fitting

The `nlsq_fit()` function uses NLSQ 0.6.0 for fast point estimates.

In [ ]:
from xpcsviewer.fitting import nlsq_fit
from xpcsviewer.fitting.models import single_exp_func

nlsq_result = nlsq_fit(
    model_fn=single_exp_func,
    x=t,
    y=g2_noise,
    yerr=g2_err,
    p0={"tau": 1.0, "baseline": 1.0, "contrast": 0.3},
    bounds={
        "tau": (1e-6, 1e6),
        "baseline": (0.5, 2.0),
        "contrast": (0.0, 1.0),
    },
)

print(f"Converged: {nlsq_result.converged}")
print(f"\nFitted parameters:")
for name, val in nlsq_result.params.items():
    std = nlsq_result.get_param_uncertainty(name)
    print(f"  {name:>10} = {val:.6f} +/- {std:.6f}")

In [ ]:
# Print the full summary
print(nlsq_result.summary())

## 3. Visualize NLSQ Fit

In [ ]:
# Generate fit curve
t_fine = np.logspace(-2, 3, 500)
g2_fit = single_exp_func(
    t_fine,
    nlsq_result.params['tau'],
    nlsq_result.params['baseline'],
    nlsq_result.params['contrast'],
)

# Prediction intervals
pi_lower, pi_upper = nlsq_result.get_prediction_interval(t_fine)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fit plot
ax = axes[0]
ax.errorbar(t, g2_noise, yerr=g2_err, fmt='o', ms=3, capsize=2, label='Data')
ax.plot(t_fine, g2_fit, 'r-', linewidth=2, label='NLSQ fit')
if not np.any(np.isnan(pi_lower)):
    ax.fill_between(t_fine, pi_lower, pi_upper, alpha=0.2, color='red', label='95% PI')
ax.set_xscale('log')
ax.set_xlabel('Delay time (s)')
ax.set_ylabel(r'$g_2(\tau)$')
ax.set_title('NLSQ Fit')
ax.legend()

# Residuals
ax = axes[1]
g2_pred = single_exp_func(
    t, nlsq_result.params['tau'],
    nlsq_result.params['baseline'],
    nlsq_result.params['contrast'],
)
residuals = g2_noise - g2_pred
ax.plot(t, residuals / g2_err, 'o', ms=3)
ax.axhline(0, color='k', linestyle='--')
ax.axhline(2, color='r', linestyle=':', alpha=0.5)
ax.axhline(-2, color='r', linestyle=':', alpha=0.5)
ax.set_xscale('log')
ax.set_xlabel('Delay time (s)')
ax.set_ylabel('Normalized residuals')
ax.set_title('Residuals')

plt.tight_layout()
plt.show()

## 4. Robust NLSQ Fitting

Use presets, stability checks, and diagnostics for difficult data.

In [ ]:
robust_result = nlsq_fit(
    model_fn=single_exp_func,
    x=t, y=g2_noise, yerr=g2_err,
    p0={"tau": 1.0, "baseline": 1.0, "contrast": 0.3},
    bounds={"tau": (1e-6, 1e6), "baseline": (0.5, 2.0), "contrast": (0, 1)},
    workflow="auto_global",     # Global optimization workflow
    stability="auto",           # Automatic numerical fixes
    fallback=True,              # Fallback strategies
    compute_diagnostics=True,   # Model health diagnostics
)

print(f"Converged: {robust_result.converged}")
print(f"R-squared: {robust_result.r_squared:.6f}")

if robust_result.diagnostics is not None:
    print(f"Health score: {robust_result.health_score}/100")
    print(f"Is healthy: {robust_result.is_healthy}")
    print(f"Condition number: {robust_result.condition_number:.2f}")

## 5. Bayesian Fitting (NumPyro NUTS)

The `fit_single_exp()` function runs NLSQ warm-start followed by NumPyro NUTS.

**Note:** This requires NumPyro and JAX to be installed.

In [ ]:
try:
    from xpcsviewer.fitting import fit_single_exp

    bayes_result = fit_single_exp(
        x=t,
        y=g2_noise,
        yerr=g2_err,
        num_warmup=500,
        num_samples=1000,
        num_chains=4,
        random_seed=42,
    )

    print("Bayesian Posterior Estimates:")
    for param in ['tau', 'baseline', 'contrast']:
        mean = bayes_result.get_mean(param)
        std = bayes_result.get_std(param)
        hdi_lo, hdi_hi = bayes_result.get_hdi(param)
        print(f"  {param:>10} = {mean:.4f} +/- {std:.4f}  "
              f"94% HDI: [{hdi_lo:.4f}, {hdi_hi:.4f}]")

    print(f"\nNLSQ warm-start: {bayes_result.nlsq_init}")

    # Diagnostics (r_hat, ess_bulk, ess_tail are per-parameter dicts)
    diag = bayes_result.diagnostics
    print(f"\nConvergence Diagnostics:")
    print(f"  Converged:    {diag.converged}")
    print(f"  Divergences:  {diag.divergences}")
    if diag.bfmi is not None:
        print(f"  BFMI:         {diag.bfmi:.4f}")
    for param in ['tau', 'baseline', 'contrast']:
        rhat = diag.r_hat.get(param, float('nan'))
        ess_b = diag.ess_bulk.get(param, 0)
        ess_t = diag.ess_tail.get(param, 0)
        print(f"  {param:>10}:  R-hat={rhat:.4f}  ESS(bulk)={ess_b:>6.0f}  ESS(tail)={ess_t:>6.0f}")

    BAYES_AVAILABLE = True

except ImportError as e:
    print(f"Bayesian fitting requires NumPyro: {e}")
    print("Install with: pip install numpyro jax arviz")
    BAYES_AVAILABLE = False

In [ ]:
if BAYES_AVAILABLE:
    # Posterior distributions
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    for i, param in enumerate(['tau', 'baseline', 'contrast']):
        samples = bayes_result.get_samples(param)
        axes[i].hist(samples, bins=50, density=True, alpha=0.7, edgecolor='black')
        axes[i].axvline(bayes_result.get_mean(param), color='r',
                        linestyle='--', label=f'Mean: {bayes_result.get_mean(param):.4f}')
        hdi_lo, hdi_hi = bayes_result.get_hdi(param)
        axes[i].axvspan(hdi_lo, hdi_hi, alpha=0.2, color='blue', label='94% HDI')
        axes[i].set_xlabel(param)
        axes[i].set_ylabel('Density')
        axes[i].legend(fontsize=8)

    plt.suptitle('Posterior Distributions', fontsize=14)
    plt.tight_layout()
    plt.show()

## 6. ArviZ Diagnostics

Use ArviZ for publication-quality diagnostic plots.

In [ ]:
if BAYES_AVAILABLE and bayes_result.arviz_data is not None:
    try:
        import arviz as az

        # Trace plot
        az.plot_trace(bayes_result.arviz_data, compact=True)
        plt.tight_layout()
        plt.show()

        # Summary statistics
        print(az.summary(bayes_result.arviz_data))
    except ImportError:
        print("Install ArviZ for diagnostic plots: pip install arviz")

## 7. Model Comparison: Single vs Double Exponential

Use AIC/BIC to determine whether a second relaxation mode is justified.

In [ ]:
from xpcsviewer.fitting.models import double_exp_func

# Fit single exponential
result_single = nlsq_fit(
    model_fn=single_exp_func,
    x=t, y=g2_noise, yerr=g2_err,
    p0={"tau": 1.0, "baseline": 1.0, "contrast": 0.3},
    bounds={"tau": (1e-6, 1e6), "baseline": (0.5, 2.0), "contrast": (0, 1)},
)

# Fit double exponential
result_double = nlsq_fit(
    model_fn=double_exp_func,
    x=t, y=g2_noise, yerr=g2_err,
    p0={"tau1": 1.0, "tau2": 50.0, "baseline": 1.0,
        "contrast1": 0.2, "contrast2": 0.1},
    bounds={
        "tau1": (1e-6, 1e4), "tau2": (1e-6, 1e6),
        "baseline": (0.5, 2.0),
        "contrast1": (0, 1), "contrast2": (0, 1),
    },
)

print("Model Comparison:")
print(f"{'':>20} {'Single Exp':>14} {'Double Exp':>14}")
print(f"{'-'*50}")
print(f"{'R-squared':>20} {result_single.r_squared:>14.6f} {result_double.r_squared:>14.6f}")
print(f"{'RMSE':>20} {result_single.rmse:>14.6e} {result_double.rmse:>14.6e}")
print(f"{'AIC':>20} {result_single.aic:>14.2f} {result_double.aic:>14.2f}")
print(f"{'BIC':>20} {result_single.bic:>14.2f} {result_double.bic:>14.2f}")
print(f"{'N parameters':>20} {len(result_single.params):>14} {len(result_double.params):>14}")

preferred = 'single' if result_single.bic < result_double.bic else 'double'
delta_bic = abs(result_single.bic - result_double.bic)
print(f"\nPreferred model: {preferred} exponential (delta BIC = {delta_bic:.1f})")

if delta_bic < 2:
    print("Evidence: Weak (delta BIC < 2)")
elif delta_bic < 6:
    print("Evidence: Positive (2 < delta BIC < 6)")
elif delta_bic < 10:
    print("Evidence: Strong (6 < delta BIC < 10)")
else:
    print("Evidence: Very strong (delta BIC > 10)")

## 8. Comparison Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Data
ax.errorbar(t, g2_noise, yerr=g2_err, fmt='o', ms=3, capsize=2,
            color='gray', label='Data', zorder=1)

# Single exp fit
g2_single = single_exp_func(
    t_fine,
    result_single.params['tau'],
    result_single.params['baseline'],
    result_single.params['contrast'],
)
ax.plot(t_fine, g2_single, 'r-', linewidth=2,
        label=f'Single exp (BIC={result_single.bic:.1f})', zorder=2)

# Double exp fit
g2_double = double_exp_func(
    t_fine,
    result_double.params['tau1'],
    result_double.params['tau2'],
    result_double.params['baseline'],
    result_double.params['contrast1'],
    result_double.params['contrast2'],
)
ax.plot(t_fine, g2_double, 'b--', linewidth=2,
        label=f'Double exp (BIC={result_double.bic:.1f})', zorder=2)

ax.set_xscale('log')
ax.set_xlabel('Delay time (s)')
ax.set_ylabel(r'$g_2(\tau)$')
ax.set_title('Model Comparison')
ax.legend()
plt.tight_layout()
plt.show()

## 9. Export Results

Save results for reproducibility.

In [ ]:
import json


class NumpyEncoder(json.JSONEncoder):
    """Handle numpy/JAX types in JSON serialization."""
    def default(self, obj):
        if hasattr(obj, 'item'):  # numpy/JAX scalar
            return obj.item()
        if hasattr(obj, 'tolist'):  # numpy/JAX array
            return obj.tolist()
        return super().default(obj)


# Export NLSQ result
export_data = {
    "method": "NLSQ",
    "model": "single_exp",
    "converged": bool(nlsq_result.converged),
    "params": {k: float(v) for k, v in nlsq_result.params.items()},
    "uncertainties": {
        name: float(nlsq_result.get_param_uncertainty(name))
        for name in nlsq_result.params
    },
    "r_squared": float(nlsq_result.r_squared),
    "rmse": float(nlsq_result.rmse),
    "aic": float(nlsq_result.aic),
    "bic": float(nlsq_result.bic),
}

print(json.dumps(export_data, indent=2, cls=NumpyEncoder))

# Optionally save Bayesian result too
if BAYES_AVAILABLE:
    bayes_dict = bayes_result.to_dict()
    print(f"\nBayesian result keys: {list(bayes_dict.keys())}")
    print(f"Package versions: {bayes_dict.get('versions', {})}")

## Summary

| Method | Speed | Uncertainty | When to Use |
|--------|-------|-------------|-------------|
| NLSQ (fast preset) | Very fast | Point estimates + CI | Quick exploration, batch processing |
| NLSQ (robust preset) | Fast | Point estimates + CI + diagnostics | Difficult data, initial screening |
| Bayesian NUTS | Slower | Full posterior + HDI + ArviZ | Publication results, model validation |

Key diagnostics to check:
- **NLSQ**: R-squared > 0.99, health score > 70, condition number < 1e6
- **Bayesian**: R-hat < 1.01, ESS > 400, divergences = 0, BFMI > 0.3